## Launch the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [43]:
import os
import shutil
import errno
import numpy as np
from pathlib import Path
import subprocess
import datetime

#### Create an executable file of STEMMUS_SCOPE

In [3]:
# path to executable
root_path_to_model = Path(Path.home(),"EcoExtreML", "STEMMUS_SCOPE")
# submit slurm job
result = subprocess.run(["sbatch", Path(root_path_to_model, "exe", "build_stemmus_scope_exe.sh")])
result.check_returncode()

success!


#### Update/set config files

In [8]:
# the user must provide the correct path
path_dict = {
    "SoilPropertyPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/SoilProperty/",
    "InputPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/",
    "OutputPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/output/",
    "ForcingPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/",
    "ForcingFileName": "AU-DaS_2010-2017_OzFlux_Met.nc",
    "VegetationPropertyPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/model_parameters/vegetation_property/",
    "DurationSize": "NA"
}

# generate a text file
root_path = Path().resolve().parent

with open(Path(root_path, "config_file_snellius.txt"), 'w') as f:
    print("The listed paths are defined in the config file:")
    for i in path_dict.keys():
        print(i)
        f.write((i + "=" + path_dict[i] + "\n"))

The listed paths are defined in the config file:
SoilPropertyPath
InputPath
OutputPath
ForcingPath
ForcingFileName
DurationSize


#### Create input directories, prepare input files 

In [55]:
for ncfile in Path(path_dict["ForcingPath"]).iterdir():
    # get the station name
    station_name = ncfile.stem.split('_')[0]
    # get start time with the format Y-M-D-HM
    start_time = str(datetime.datetime.now()).split('.')[0]
    timestamp = start_time.replace(' ', '-').replace(':', '')[:-2]
    # create input directory
    work_dir = Path(path_dict["InputPath"], station_name + '_' + timestamp)
    Path(work_dir).mkdir(parents=True, exist_ok=True)
    print(f"Prepare work directory for the station:{station_name}")
    # copy model parameters to work directory
    shutil.copytree(path_dict["VegetationPropertyPath"], work_dir, dirs_exist_ok=True)
    # update config file for ForcingFileName and InputPath
    
    with open(Path(work_dir, f"{station_name}_{timestamp}_config.txt"), 'w') as f:
        for i in path_dict.keys():
            if i == "ForcingFileName":
                f.write((i + "=" + ncfile.stem + ".nc" + "\n"))
            elif i == "InputPath":
                f.write((i + "=" + str(work_dir) + "\n"))
            else:
                f.write((i + "=" + path_dict[i] + "\n"))


Prepare work directory for the station:nwe-3
Prepare work directory for the station:alp-3
Prepare work directory for the station:neu-3
Prepare work directory for the station:cee-3
Prepare work directory for the station:ceu-3
Prepare work directory for the station:swe-3
Prepare work directory for the station:see-3


#### Run the model

#### Create output directories, prepare output files